In [0]:
import pandas as pd

import pickle


import dask.dataframe as dd
df = dd.read_csv("../Wavenumber/*").compute().reset_index()

# df = pd.read_csv("/content/drive/My Drive/Year4Project/*")

X = df.iloc[:,:400]
Y = df.iloc[:,400:]



In [61]:
Y.head()

,feature_400,label_1,label_2,label_3,label_4,label_5,label_6,label_7
0,0.000001,c,-2,-2,-2,1.178403,1.987752,1.231197
1,0.000001,e,0,0,0,1.802624,0.778109,1.407177
2,0.000001,c,-2,-2,-2,1.294944,0.763598,1.651345
3,0.000001,e,0,0,0,1.921284,0.826330,1.249305
4,0.000001,c,-2,-2,-2,0.810580,0.784436,0.944912


In [62]:
X.head()

,index,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_360,feature_361,feature_362,feature_363,feature_364,feature_365,feature_366,feature_367,feature_368,feature_369,feature_370,feature_371,feature_372,feature_373,feature_374,feature_375,feature_376,feature_377,feature_378,feature_379,feature_380,feature_381,feature_382,feature_383,feature_384,feature_385,feature_386,feature_387,feature_388,feature_389,feature_390,feature_391,feature_392,feature_393,feature_394,feature_395,feature_396,feature_397,feature_398,feature_399
0,-117.413551,-117.414613,-117.417829,-117.423202,-117.430734,-117.440428,-117.452289,-117.466324,-117.482539,-117.500943,-117.521544,-117.544354,-117.569383,-117.596645,-117.626154,-117.657926,-117.691975,-117.728322,-117.766984,-117.807983,-117.851341,-117.897081,-117.945229,-117.995812,-118.048858,-118.104397,-118.162461,-118.223085,-118.286305,-118.352158,-118.420684,-118.491927,-118.565931,-118.642744,-118.722416,-118.804999,-118.890550,-118.979128,-119.070795,-119.165617,...,-119.166952,-119.072086,-118.980376,-118.891756,-118.806164,-118.723540,-118.643828,-118.566976,-118.492933,-118.421652,-118.353088,-118.287199,-118.223943,-118.163283,-118.105184,-118.049610,-117.996530,-117.945913,-117.897732,-117.851958,-117.808567,-117.767536,-117.728841,-117.692463,-117.658381,-117.626579,-117.597039,-117.569745,-117.544685,-117.521845,-117.501213,-117.482779,-117.466534,-117.452469,-117.440577,-117.430853,-117.423292,-117.417889,-117.414643,-117.413551
1,-113.881322,-113.882395,-113.885622,-113.891006,-113.898548,-113.908253,-113.920126,-113.934171,-113.950398,-113.968812,-113.989425,-114.012246,-114.037287,-114.064560,-114.094081,-114.125863,-114.159925,-114.196283,-114.234957,-114.275968,-114.319338,-114.365090,-114.413251,-114.463846,-114.516904,-114.572456,-114.630533,-114.691170,-114.754403,-114.820269,-114.888810,-114.960067,-115.034085,-115.110912,-115.190598,-115.273197,-115.358763,-115.447356,-115.539039,-115.633877,...,-115.634237,-115.539387,-115.447693,-115.359088,-115.273511,-115.190901,-115.111204,-115.034366,-114.960338,-114.889071,-114.820520,-114.754644,-114.691401,-114.630755,-114.572668,-114.517107,-114.464039,-114.413435,-114.365265,-114.319504,-114.276125,-114.235106,-114.196423,-114.160056,-114.125986,-114.094195,-114.064666,-114.037384,-114.012335,-113.989506,-113.968885,-113.950462,-113.934228,-113.920174,-113.908293,-113.898580,-113.891030,-113.885638,-113.882403,-113.881322
2,-123.985723,-123.986806,-123.990043,-123.995437,-124.002989,-124.012704,-124.024586,-124.038642,-124.054878,-124.073302,-124.093925,-124.116756,-124.141807,-124.169090,-124.198621,-124.230414,-124.264486,-124.300854,-124.339539,-124.380560,-124.423941,-124.469704,-124.517875,-124.568482,-124.621551,-124.677114,-124.735203,-124.795852,-124.859096,-124.924975,-124.993527,-125.064797,-125.138827,-125.215667,-125.295367,-125.377978,-125.463558,-125.552165,-125.643862,-125.738714,...,-125.738200,-125.643365,-125.551684,-125.463094,-125.377530,-125.294934,-125.215250,-125.138425,-125.064409,-124.993155,-124.924616,-124.858752,-124.795522,-124.734887,-124.676811,-124.621261,-124.568205,-124.517612,-124.469454,-124.423703,-124.380335,-124.339326,-124.300654,-124.264298,-124.230238,-124.198457,-124.168939,-124.141667,-124.116628,-124.093809,-124.073198,-124.054785,-124.038561,-124.024517,-124.012646,-124.002943,-123.995402,-123.990020,-123.986794,-123.985723
3,-114.424147,-114.425230,-114.428468,-114.433861,-114.441414,-114.451129,-114.463012,-114.477068,-114.493304,-114.511729,-114.532352,-114.555183,-114.580235,-114.607519,-114.637050,-114.6688

In [63]:
Y.columns = ['k', 'shape', 'origin1', 'origin2', 'origin3', 'length1', 'length2', 'length3']

Y.head()

,k,shape,origin1,origin2,origin3,length1,length2,length3
0,0.000001,c,-2,-2,-2,1.178403,1.987752,1.231197
1,0.000001,e,0,0,0,1.802624,0.778109,1.407177
2,0.000001,c,-2,-2,-2,1.294944,0.763598,1.651345
3,0.000001,e,0,0,0,1.921284,0.826330,1.249305
4,0.000001,c,-2,-2,-2,0.810580,0.784436,0.944912


In [0]:
import numpy as np


gridpoints = []
for z in np.linspace(-2,2,11):
  for y in np.linspace(-2,2,11):
    for x in np.linspace(-2,2,11):
      gridpoints.append((x,y,z))


In [0]:
def insideEllipsoid(radii, point):
  a = radii[0]
  b = radii[1]
  c = radii[2]

  x = point[0]
  y = point[1]
  z = point[2]

  return (x/a)**2 + (y/c)**2 + (z/c)**2 < 1

def insideCuboid(point, y):

  l0 = y[0]
  l1 = y[1]
  l2 = y[2]

  bool0 = (point[0]>-l0/2) & (point[0]<l0/2)
  bool1 = (point[1]>-l1/2) & (point[1]<l1/2)
  bool2 = (point[2]>-l2/2) & (point[2]<l2/2)

  return bool0 & bool1 & bool2

def toVoxels(datapoint, gridpoints):
  if datapoint['shape']=='e':
    return np.array([insideEllipsoid(datapoint[['length1', 'length2', 'length3']].values, vox)  for vox in gridpoints])

  if datapoint['shape']=='c':
    return np.array([insideCuboid(datapoint[['length1', 'length2', 'length3']].values, vox)  for vox in gridpoints])


Y1 = np.array([toVoxels(y, gridpoints) for index, y in Y.iterrows()])

In [0]:
pd.concat([X, Y['k'], pd.DataFrame(Y1)], axis=1).to_csv('../VoxelsDataset/wavenumberVoxels.csv')

In [0]:
import dask.dataframe as dd
dd.read_csv("../VoxelsDataset/wavenumberVoxels/*").compute()

In [11]:
df.head()

,index,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330
0,-117.413551,-117.414613,-117.417829,-117.423202,-117.430734,-117.440428,-117.452289,-117.466324,-117.482539,-117.500943,-117.521544,-117.544354,-117.569383,-117.596645,-117.626154,-117.657926,-117.691975,-117.728322,-117.766984,-117.807983,-117.851341,-117.897081,-117.945229,-117.995812,-118.048858,-118.104397,-118.162461,-118.223085,-118.286305,-118.352158,-118.420684,-118.491927,-118.565931,-118.642744,-118.722416,-118.804999,-118.890550,-118.979128,-119.070795,-119.165617,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,-113.881322,-113.882395,-113.885622,-113.891006,-113.898548,-113.908253,-113.920126,-113.934171,-113.950398,-113.968812,-113.989425,-114.012246,-114.037287,-114.064560,-114.094081,-114.125863,-114.159925,-114.196283,-114.234957,-114.275968,-114.319338,-114.365090,-114.413251,-114.463846,-114.516904,-114.572456,-114.630533,-114.691170,-114.754403,-114.820269,-114.888810,-114.960067,-115.034085,-115.110912,-115.190598,-115.273197,-115.358763,-115.447356,-115.539039,-115.633877,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,-123.985723,-123.986806,-123.990043,-123.995437,-124.002989,-124.012704,-124.024586,-124.038642,-124.054878,-124.073302,-124.093925,-124.116756,-124.141807,-124.169090,-124.198621,-124.230414,-124.264486,-124.300854,-124.339539,-124.380560,-124.423941,-124.469704,-124.517875,-124.568482,-124.621551,-124.677114,-124.735203,-124.795852,-124.859096,-124.924975,-124.993527,-125.064797,-125.138827,-125.215667,-125.295367,-125.377978,-125.463558,-125.552165,-125.643862,-125.738714,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,-114.424147,-114.425230,-114.428468,-114.433861,-114.441414,-114.451129,-114.463012,-114.477068,-114.493304,-114.511729,-114.532352,-114.555183,-114.580235,-114.607519,-114.637050,-114.668843,-114.702915,-114.739284,-114.777969,-114.818991,-114.862372,-114.908136,-114.956308,-115.006914,-115.059984,-115.115548,-115.173637,-115.234286,-115.297531,-115.363410,-115.431963,-115.503233,-115.577264,-115.654105,-115.733805,-115.816417,-115.901997,-115.990605,-116.082302,-116.177155,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,-131.450275,-131.451352,-131.454583,-131.459971,-131.467517,-131.477226,-131.489102,-131.503152,-131.519382,-131.537801,-131.558417,-131.581242,-131.606287,-131.633564,-131.663089,-131.694875,-131.728941,-131.765303,-131.803981,-131.844996,-131.888370,-131.934127,-131.982292,-132.032891,-132.085954,-132.141510,-132.199592,-132.260234,-132.323471,-132.389342,-132.457887,-132.529149,-132.603172,-132.680004,-132.759696,-132.842299,-132.927871,-133.016469,-133.10

In [0]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:401]
Y = df.iloc[:,401:]

x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.7, random_state = 42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.7, random_state = 42)


In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_val = ss.transform(x_val)
x_test = ss.transform(x_test)


In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD, Adam


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=50, verbose=1,)
early_stop = EarlyStopping(monitor='val_loss', patience=61, verbose=1, restore_best_weights=True)
opt = Adam()

model = Sequential()
model.add(Dense(1000, input_dim=401, kernel_initializer='normal', activation='relu'))
model.add(Dense(2000, kernel_initializer='normal', activation='relu'))
model.add(Dense(2000, kernel_initializer='normal', activation='relu'))
model.add(Dense(2000, kernel_initializer='normal', activation='relu'))
model.add(Dense(2000, kernel_initializer='normal', activation='relu'))
model.add(Dense(2000, kernel_initializer='normal', activation='relu'))
model.add(Dense(1331, kernel_initializer='normal', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'], )
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 1000)              402000    
_________________________________________________________________
dense_17 (Dense)             (None, 2000)              2002000   
_________________________________________________________________
dense_18 (Dense)             (None, 2000)              4002000   
_________________________________________________________________
dense_19 (Dense)             (None, 2000)              4002000   
_________________________________________________________________
dense_20 (Dense)             (None, 2000)              4002000   
_________________________________________________________________
dense_21 (Dense)             (None, 2000)              4002000   
_________________________________________________________________
dense_22 (Dense)             (None, 1331)             

In [17]:
history = model.fit(x=x_train, y=y_train, validation_data = (x_val, y_val), epochs=1000, verbose=1, callbacks=[early_stop, reduce_lr])

Train on 1523 samples, validate on 654 samples
Epoch 1/1000
1523/1523 [==============================] - 19s 13ms/step - loss: 0.2932 - acc: 0.9161 - val_loss: 0.1546 - val_acc: 0.9292
Epoch 2/1000
1523/1523 [==============================] - 18s 12ms/step - loss: 0.1496 - acc: 0.9362 - val_loss: 0.1556 - val_acc: 0.9305
Epoch 3/1000
1523/1523 [==============================] - 18s 12ms/step - loss: 0.1458 - acc: 0.9370 - val_loss: 0.1583 - val_acc: 0.9347
Epoch 4/1000
1523/1523 [==============================] - 18s 12ms/step - loss: 0.1473 - acc: 0.9374 - val_loss: 0.1480 - val_acc: 0.9335
Epoch 5/1000
1523/1523 [==============================] - 18s 12ms/step - loss: 0.1395 - acc: 0.9386 - val_loss: 0.1433 - val_acc: 0.9358
Epoch 6/1000
1523/1523 [==============================] - 18s 12ms/step - loss: 0.1365 - acc: 0.9397 - val_loss: 0.1430 - val_acc: 0.9350
Epoch 7/1000
1523/1523 [==============================] - 18s 12ms/step - loss: 0.1382 - acc: 0.9387 - val_loss: 0.1455 - val

In [87]:
X.head()

,index,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_361,feature_362,feature_363,feature_364,feature_365,feature_366,feature_367,feature_368,feature_369,feature_370,feature_371,feature_372,feature_373,feature_374,feature_375,feature_376,feature_377,feature_378,feature_379,feature_380,feature_381,feature_382,feature_383,feature_384,feature_385,feature_386,feature_387,feature_388,feature_389,feature_390,feature_391,feature_392,feature_393,feature_394,feature_395,feature_396,feature_397,feature_398,feature_399,k
0,-117.413551,-117.414613,-117.417829,-117.423202,-117.430734,-117.440428,-117.452289,-117.466324,-117.482539,-117.500943,-117.521544,-117.544354,-117.569383,-117.596645,-117.626154,-117.657926,-117.691975,-117.728322,-117.766984,-117.807983,-117.851341,-117.897081,-117.945229,-117.995812,-118.048858,-118.104397,-118.162461,-118.223085,-118.286305,-118.352158,-118.420684,-118.491927,-118.565931,-118.642744,-118.722416,-118.804999,-118.890550,-118.979128,-119.070795,-119.165617,...,-119.072086,-118.980376,-118.891756,-118.806164,-118.723540,-118.643828,-118.566976,-118.492933,-118.421652,-118.353088,-118.287199,-118.223943,-118.163283,-118.105184,-118.049610,-117.996530,-117.945913,-117.897732,-117.851958,-117.808567,-117.767536,-117.728841,-117.692463,-117.658381,-117.626579,-117.597039,-117.569745,-117.544685,-117.521845,-117.501213,-117.482779,-117.466534,-117.452469,-117.440577,-117.430853,-117.423292,-117.417889,-117.414643,-117.413551,0.000001
1,-113.881322,-113.882395,-113.885622,-113.891006,-113.898548,-113.908253,-113.920126,-113.934171,-113.950398,-113.968812,-113.989425,-114.012246,-114.037287,-114.064560,-114.094081,-114.125863,-114.159925,-114.196283,-114.234957,-114.275968,-114.319338,-114.365090,-114.413251,-114.463846,-114.516904,-114.572456,-114.630533,-114.691170,-114.754403,-114.820269,-114.888810,-114.960067,-115.034085,-115.110912,-115.190598,-115.273197,-115.358763,-115.447356,-115.539039,-115.633877,...,-115.539387,-115.447693,-115.359088,-115.273511,-115.190901,-115.111204,-115.034366,-114.960338,-114.889071,-114.820520,-114.754644,-114.691401,-114.630755,-114.572668,-114.517107,-114.464039,-114.413435,-114.365265,-114.319504,-114.276125,-114.235106,-114.196423,-114.160056,-114.125986,-114.094195,-114.064666,-114.037384,-114.012335,-113.989506,-113.968885,-113.950462,-113.934228,-113.920174,-113.908293,-113.898580,-113.891030,-113.885638,-113.882403,-113.881322,0.000001
2,-123.985723,-123.986806,-123.990043,-123.995437,-124.002989,-124.012704,-124.024586,-124.038642,-124.054878,-124.073302,-124.093925,-124.116756,-124.141807,-124.169090,-124.198621,-124.230414,-124.264486,-124.300854,-124.339539,-124.380560,-124.423941,-124.469704,-124.517875,-124.568482,-124.621551,-124.677114,-124.735203,-124.795852,-124.859096,-124.924975,-124.993527,-125.064797,-125.138827,-125.215667,-125.295367,-125.377978,-125.463558,-125.552165,-125.643862,-125.738714,...,-125.643365,-125.551684,-125.463094,-125.377530,-125.294934,-125.215250,-125.138425,-125.064409,-124.993155,-124.924616,-124.858752,-124.795522,-124.734887,-124.676811,-124.621261,-124.568205,-124.517612,-124.469454,-124.423703,-124.380335,-124.339326,-124.300654,-124.264298,-124.230238,-124.198457,-124.168939,-124.141667,-124.116628,-124.093809,-124.073198,-124.054785,-124.038561,-124.024517,-124.012646,-124.002943,-123.995402,-123.990020,-123.986794,-123.985723,0.000001
3,-114.424147,-114.425230,-114.428468,-114.433861,-114.441414,-114.451129,-114.463012,-114.477068,-114.493304,-114.511729,-114.532352,-114.555183,-114.580235,-114.607519,-114.637050,-114.668843,-114.702915,-114

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
